# Deploying a Python Function in CP4D
Example notebook showing how to create a Python function and then store it as an asset in CP4D which can be invoked via REST API

In [ ]:
import os

In [ ]:
#Function example: Connects to DVM for z/OS, executes a SQL statement, and returns the result set. 

def my_deployable_function():
    
    # put imports here (only imported once when the deployment is created)
    import itc_utils.flight_service as itcfs


    def score(payload):
        
        # Select the schema from the payload. An array of arrays is expected as input to 
        schema_from_payload = payload.get("input_data")[0].get("values")[0][0]
        response_message = "Received message - {0}".format(schema_from_payload)
        
        
        ###### Below code block was generated by creating a Data Ingestion Code Snippet to read data from a platform connection ##########
        readClient = itcfs.get_flight_client()


        DVM_for_z_OS_data_request = {
            'connection_name': """DVM for z/OS""",
            'interaction_properties': {
                'row_limit': 5000,
                'schema_name': 'DVSQL',
                'table_name': schema_from_payload
            }
        }

        flightInfo = itcfs.get_flight_info(readClient, nb_data_request=DVM_for_z_OS_data_request)
        
        # put the result set in a data frame
        data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
        data_df_1.head(10)
        
        # json-ify the data frame
        response_message = data_df_1.to_json()

        # return the response
        # must use the 'predictions' key
        score_response = {
            'predictions': [{'fields': ['Response_message_field'],
                             'values': [[response_message]]
                            }]
        }
        return score_response

    return score


In [ ]:
score = my_deployable_function()

In [ ]:
# Need to have the 'input_data' and 'values' keys, and need to have the values be an 'array of arrays'
schema = {
    "input_data": [{
        "values" : [["CUSTOMER01"]]
    }]
}

In [ ]:
function_result = score(schema)

In [ ]:
function_result

## Connecting to WML

In [ ]:
wml_credentials = {
    "token": os.environ['USER_ACCESS_TOKEN']#API Auth Token ,
    "instance_id": "openshift",
    "url": os.environ['RUNTIME_ENV_APSX_URL'],
    "version": "4.7" 
}


In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

In [ ]:
client.spaces.list(limit=10)

In [ ]:
client.set.default_space('1924fc56-cd19-4784-ba52-b893cd987696')

If you don't have a Deployment Space:

In [ ]:

# space_name ="My Deployment Space"
# space_id = client.spaces.store(meta_props={client.spaces.ConfigurationMetaNames.NAME: space_name})['metadata']['id']
# client.set.default_space(space_id)

## Storing the Function 

In [ ]:
sw_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.2-py3.10")

In [ ]:
function_meta = {
    client.repository.FunctionMetaNames.NAME:"test_function",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: sw_spec_uid
}

In [ ]:
func_details = client.repository.store_function(my_deployable_function, function_meta)

## Deploying the Function 

In [ ]:
function_uid = client.repository.get_model_id(func_details)

In [ ]:
deployment_details = client.deployments.create(
    function_uid, 
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: 'Python Function Deployment',
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = client.deployments.get_scoring_href(deployment_details)
deployment_uid=client.deployments.get_id(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(function_uid))
print("Deployment id: {}".format(deployment_uid))



## Calling the Python Function Deployment from External Clients

In [ ]:
import requests
import json

In [ ]:
# Get an Auth Token



#  Or run a curl command to get the token: https://www.ibm.com/docs/en/cloud-paks/cp-data/4.7.x?topic=deployments-authenticating-programmatic-access
#
#  !curl -k -X POST https://cpd-cp4d.apps.z1fd.dmz/icp4d-api/v1/authorize -H 'cache-control: no-cache' -H 'content-type: application/json' -d '{"username":"aaminin","password":"<password>"}'


cp4d_url = "https://cpd-cp4d.apps.z1fd.dmz/"
cp4d_auth_url = cp4d_url + "icp4d-api/v1/authorize"

headers = {
    "cache-control": "no-cache",
    "Content-type" : "application/json"
}
credentials = {"username":"aaminin","password":"checkers"}

In [ ]:
response = requests.post(cp4d_auth_url, json=credentials,verify=False,headers=headers)#cert_path,cert=cert_path)

if not response.ok:
    raise Exception(str(response.content))

# Convert response to dict
response_json = json.loads(response.text)
token = response_json['token']

# In watson studio: token = os.environ['USER_ACCESS_TOKEN']

Invoke the endpoint

In [ ]:
headers = {
    "cache-control": "no-cache",
    "Content-type" : "application/json",
    "Authorization": "Bearer "+ token,
}

schema = {
    "input_data": [{
        "values" : [["CUSTOMER01"]]
    }]
}

# In this example we use the saved cluster local endpoint
# The external endpoint would be something like: https://<base_cp4d_url>/ml/v4/deployments/9424a606-b591-4379-98b9-024a2ebff1cc/predictions

response = requests.post(scoring_url,json=schema,verify=False,headers=headers)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
response_json